# Integración Final: Modelado en Capas, Modelado de Datos y Práctica Completa



En esta sesión integraremos todo lo aprendido durante el curso:
 - Arquitectura de datos en capas (Bronze, Silver, Gold)
 - Modelado de datos en Databricks con Delta Lake
 - Automatización y orquestación con Workflows

## Parte 1: Arquitectura en Capas

En Databricks, el **modelo en capas** es una práctica recomendada para estructurar los datos:

### 1. **Bronze** (Capa cruda)
- Datos tal cual vienen del origen.
- Poco o ningún procesamiento.
- Se usa para trazabilidad y auditoría.

### 2. **Silver** (Capa limpia)
- Datos depurados, con tipos bien definidos.
- Se aplican validaciones y joins si es necesario.
- Se preparan para análisis y agregaciones.

### 3. **Gold** (Capa de negocio)
- Métricas clave, KPIs.
- Datos listos para dashboards o consumo analítico.

Cada capa se almacena en tablas Delta y puede tener gobernanza a través de Unity Catalog.


## Parte 2: Modelado de Datos

Para este caso, usamos una variante del **modelo en estrella (Star Schema)**:

- **Tabla de hechos**: vuelos y mantenimientos
- **Tablas de dimensión**: aeropuertos, aeronaves

Este modelo nos permite hacer análisis por rutas, retrasos por aeropuerto, rendimiento por aerolínea, etc.

Ejemplo de dimensiones:
- `dim_aeropuertos (aeropuerto_id, nombre, ciudad, pais, lat, lon)`
- `dim_aeronaves (aeronave_id, modelo, fabricante, anio_fabricacion)`

Ejemplo de hecho:
- `fact_vuelos (vuelo_id, fecha, origen_id, destino_id, aeronave_id, estado, duracion_min)`
- `fact_mantenimientos (mantenimiento_id, aeronave_id, fecha, tipo, costo_usd, duracion_hr)`


## Parte 3: Práctica Final

### Objetivo
Crear un pipeline de datos (Workflow) partiendo de los ficheros csv que tienen que ser ingestado en `bronce`, hacer una serie de validaciones para cargar estos datos posteriormente en `plata`, y por último crear 2 procesos independientes para generar 2 tablas en `oro`: una calculando un KPI de puntualidad por mes y modelo de aeronave y otro proceso para calcular osto promedio de mantenimiento por tipo y modelo de aeronave

### Archivos utilizados
- `aerolineas.csv`
- `aeropuertos.csv`
- `vuelos.csv`
- `mantenimientos.csv`

### Pasos:
0. Crear un workflow
1. Ingesta de datos crudos a capa **bronce**. (*Nota en la siguiente celda*):
    - Cargar los datos en el `Catalog`
    - Crear un script `.py` que sea parametrizable para cargar los datos desde los ficheros a tablas delta.
    - Este script tiene que tener 4 parametros: `input_path`, `output_path`, `write_mode`, `partition_by`
    - Crear tareas para ingestar los archivos utilizando los puntos anteriores
2. Limpieza y relaciones en **plata**
    - Revisar el fichero `config.json`
    - Crear un script de validación de datos que lea de una tabla delta, que compruebe los puntos del fichero anterior y escriba en la capa `plata`. 
    - Este script tiene que ser parametrizable con los siguientes argumentos: `input_table`, `output_table`, `config_path`, `table_name`
3. Generación de métricas en **oro**
    - Crear dos script uno para cada proceso
    - Analizar los datos de los ficheros y comprobar cómo podría llevarse acabo


*Nota*: La idea original era utilizar una fuente de datos externa e ingestarla. Esto se podía realizar de dos formas:
 - Mediante un ingestion pipeline pero daba error por permisos
 - Mediante un script consultando una fuente de datos pero el cluster no tiene salida a internet

In [0]:
import requests
r = requests.get("https://databricks-ceste.s3.eu-west-1.amazonaws.com/mantenimientos_rds.csv")
print(r.status_code)

## Puntos fuertes de este proceso

1. Ingesta de múltiples orígenes
2. El modelado de capas facilita el debug, permite reusabilidad y versionamiento de datos intermedios y escalar a nuevos pipeline sin duplicar código
3. La orquestación con workflow permite automatización 

## Pasos Opcionales

1. Ingestar otro fichero para las tablas de hechos y para las de dimensiones y ver la diferencia (append /overwrite)
2. Automatizar nueva llegada de ficheros
3. Diseño de logs
4. Creacion de permisos con Unity Catalog